In [1]:
from yfpy.query import YahooFantasySportsQuery
from yfpy.models import YahooFantasyObject, Player, StatCategories
from datetime import datetime, timedelta, timezone
from dotenv import load_dotenv
import os
from nhlpy import NHLClient
from yfpy.utils import reformat_json_list, unpack_data, prettify_data, jsonify_data
import requests
import json
from zoneinfo import ZoneInfo
from collections import defaultdict
import unicodedata
from pathlib import Path
import xml.etree.ElementTree as ET
import math

In [ ]:
import requests
import webbrowser
import time
import subprocess

def open_browser_in_incognito(url):
    subprocess.run(['open', '-a', 'Safari', '--args', '--private', url])

# --------------- Step 1: Configure App Credentials ---------------- #
load_dotenv()
CLIENT_ID = os.getenv("YAHOO_CONSUMER_KEY")
CLIENT_SECRET = os.getenv("YAHOO_CONSUMER_SECRET")
REDIRECT_URI = os.getenv("REDIRECT_URI")
AUTH_URL = "https://api.login.yahoo.com/oauth2/request_auth"
TOKEN_URL = "https://api.login.yahoo.com/oauth2/get_token"

# --------------- Step 2: Get Authorization Code ---------------- #
def get_authorization_code():
    """
    Redirect user to Yahoo login page to obtain authorization code.
    """
    print("Client ID:", CLIENT_ID)
    print("Redirect URI:", REDIRECT_URI)
    auth_params = {
        "client_id": CLIENT_ID,
        "redirect_uri": REDIRECT_URI,
        "response_type": "code",
    }
    auth_request_url = f"{AUTH_URL}?client_id={auth_params['client_id']}&redirect_uri={auth_params['redirect_uri']}&response_type={auth_params['response_type']}"
    
    print("Opening the browser to authenticate with Yahoo...")
    open_browser_in_incognito(auth_request_url)
    # webbrowser.open(auth_request_url)

    auth_code = input("Enter the authorization code from the URL after login: ").strip()
    print(auth_code)
    return auth_code


# --------------- Step 3: Exchange Authorization Code for Access Token ---------------- #
def get_access_token(auth_code):
    """
    Exchange authorization code for access token and refresh token.
    """
    token_data = {
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "redirect_uri": REDIRECT_URI,
        "code": auth_code,
        "grant_type": "authorization_code",
    }

    response = requests.post(TOKEN_URL, data=token_data)
    token_info = response.json()
    
    if "access_token" not in token_info:
        print("Error obtaining access token:", token_info)
        return None

    return token_info


# --------------- Step 4: Make API Request with Access Token ---------------- #
def make_authenticated_request(access_token):
    """
    Make an API request to Yahoo Fantasy API using the access token.
    """
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json",
    }

    league_key = f"{game_id}.l.{league_id}"
    url = f"{BASE_URL}/league/{league_key}"

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print("Error making API request:", response.text)
        return None


# --------------- Step 5: Refresh Access Token ---------------- #
def refresh_access_token(refresh_token):
    """
    Refresh access token when it expires.
    """
    refresh_data = {
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "refresh_token": refresh_token,
        "grant_type": "refresh_token",
    }

    response = requests.post(TOKEN_URL, data=refresh_data)
    new_tokens = response.json()

    if "access_token" not in new_tokens:
        print("Error refreshing access token:", new_tokens)
        return None

    return new_tokens


# --------------- Main Execution ---------------- #
if __name__ == "__main__":
    auth_code = get_authorization_code()
    
    if not auth_code:
        print("Failed to retrieve authorization code. Exiting.")
        exit(1)

    token_info = get_access_token(auth_code)
    
    if not token_info:
        print("Failed to retrieve access token. Exiting.")
        exit(1)

    access_token = token_info["access_token"]
    refresh_token = token_info["refresh_token"]
    expires_in = token_info["expires_in"]

    print(f"Access Token: {access_token}\nRefresh Token: {refresh_token}\nExpires in: {expires_in} seconds")

    # Make API request with access token
    data = make_authenticated_request(access_token)
    print("API Response:", data)

    # Simulate token expiration and refresh
    print("\nWaiting for token expiration (simulated)...")
    time.sleep(5)  # Replace this with `time.sleep(expires_in)` for real expiration time

    new_tokens = refresh_access_token(refresh_token)
    if new_tokens:
        print("New Access Token:", new_tokens["access_token"])

Client ID: dj0yJmk9bU10Q0gxVE12TzdDJmQ9WVdrOVdIQTJWRGRDYURFbWNHbzlNQT09JnM9Y29uc3VtZXJzZWNyZXQmc3Y9MCZ4PWU1
Redirect URI: https://localhost:3000/api/auth/callback/yahoo
Opening the browser to authenticate with Yahoo...

Failed to retrieve authorization code. Exiting.
Error obtaining access token: {'error': 'INVALID_INPUT', 'error_description': 'code parameter cannot be empty for authorization_code grant type'}
Failed to retrieve access token. Exiting.


TypeError: 'NoneType' object is not subscriptable

In [3]:
nhl_client = NHLClient()

In [38]:
# Use last season stats so there aren't any spoilers
current = False
league_id = "67269" if current else "97108"
game_id = 453 if current else 427

In [46]:
load_dotenv() # Can remove this in production, only need for ipynb
query = YahooFantasySportsQuery(
    league_id=league_id,
    game_code="nhl",
    game_id=game_id,
)

In [ ]:
league_info = query.get_league_info()
league_start_week = league_info.start_week
league_end_week = league_info.end_week
league_start_date_str = league_info.start_date
league_end_date_str = league_info.end_date
league_teams = league_info.teams

In [ ]:
def strip_namespace(tag):
    """Remove namespace from the tag name."""
    return tag.split("}")[-1] if "}" in tag else tag

def parse_element(element):
    parsed_data = {}
    children = list(element)

    # Check if all children have the same tag (implying a list)
    child_tags = [strip_namespace(child.tag) for child in children]
    is_list = len(set(child_tags)) == 1 and len(children) > 1

    # Convert child elements into dictionary keys
    if is_list:
        return [parse_element(child) for child in children]  # Return a list directly
    else:
        for child in children:
            tag = strip_namespace(child.tag)
            child_data = parse_element(child)

            # Handle multiple children with the same tag by storing them as lists
            if tag in parsed_data:
                if not isinstance(parsed_data[tag], list):
                    parsed_data[tag] = [parsed_data[tag]]
                parsed_data[tag].append(child_data)
            else:
                parsed_data[tag] = child_data

    # If the element has text content, add it
    text = element.text.strip() if element.text else ""
    if text and not parsed_data:
        return text  # Return text if no nested structure

    return parsed_data or text  # Return text if no nested structure

def xml_to_dict(xml_string):
    root = ET.fromstring(xml_string)
    return {strip_namespace(root.tag): parse_element(root)}

In [4]:
BASE_URL = "https://fantasysports.yahooapis.com/fantasy/v2"

In [47]:
query.save_access_token_data_to_env_file(
    Path("")
)

In [48]:
load_dotenv() # Can remove this in production, only need for ipynb
ACCESS_TOKEN = "QG9CsHGdoxhqmagTYp.4dFRTCeZ74gwZZ6HaneCKkS83_Ys0gNSz83wZ0iooB_s_jTCfxgT4YLbbAWj9cqgtl6HtnbgEWRhJWMunTj1sqAB0xp0.aPCatPKojA1wuxRRbWe.1yHChIpDQULiymJnPjfE6hB4VP3zsmCkK_46t6EfAxjVAbxhFN5W2kt4a0UvSh8fQJM6vsB1XviJ2AAdsUBYvA.MkSWqoOw_KqbJ_N7H5zilhVOcyUqWaK9AAjQAlpMI8KJbGkkdpnSSi295cxId1sKfNgXNdaTcUBZWoMDgXioPzBJVmxy8OtoikAuBrEAOURQtnRW.u9qC8x2XUPb13Pl9RALDvQowooTmaur8wD954sYG7GfCooCEzOgvw1431xH8qooYblQiL8YcIW3CXnl8oPYmxmnBTFtZ1kWAWeKr0iHjHJdwC23QxLAplQk_qwDs7hTEWXaAQhLsLZB5NVpDVljB_TkpnHa1uOAUcfmblCPO9qNUDW6fgupM4IvBwvcUf.3atahFLE_5b3eHOOiOUseZVacN7XxWUML8ci_8_vnd1xPomPHvXKmGpmxeh51CVotFzvLpmpLd8NAYp5xDm25LOv16Swq3KpjKHr.XSrtf4yJj8fFJ0tzaD_uyPb1HAzGr.BBICSRcewkJbtfVgYHp1pADx2MyY8FflvE3cvvuRfX_T7PuApSpaeinEvlaMJozePwO75m8oNOej6_JdmvqpO3PSjvttccp4Mc14QZuuw3gI.Gofc_Zk6RSPKXA2gZZ7zpKFb5fZ6LaIvbgexEI68JWZGT7RrNOr5p3PNyQp6Q0o17YK8aP8C4QCaWALGhowfx5te.8sIfYi6zQyXx_90DYTFbifQ0iSMrc03DObTfxyHtKXSsfrBioaBWpI3lG1nlYMrf.vMAFgIRifzY_5d0RIzaAZWrtMUX7_bFwuLCQhiojVeTE5QWY5n8wTX2FdrHmtExAig.Z5Hc-"
# ACCESS_TOKEN = os.getenv("YAHOO_ACCESS_TOKEN")
print(ACCESS_TOKEN)
headers = {
    "Authorization": f"Bearer {ACCESS_TOKEN}",
    "Content-Type": "application/json"
}

QG9CsHGdoxhqmagTYp.4dFRTCeZ74gwZZ6HaneCKkS83_Ys0gNSz83wZ0iooB_s_jTCfxgT4YLbbAWj9cqgtl6HtnbgEWRhJWMunTj1sqAB0xp0.aPCatPKojA1wuxRRbWe.1yHChIpDQULiymJnPjfE6hB4VP3zsmCkK_46t6EfAxjVAbxhFN5W2kt4a0UvSh8fQJM6vsB1XviJ2AAdsUBYvA.MkSWqoOw_KqbJ_N7H5zilhVOcyUqWaK9AAjQAlpMI8KJbGkkdpnSSi295cxId1sKfNgXNdaTcUBZWoMDgXioPzBJVmxy8OtoikAuBrEAOURQtnRW.u9qC8x2XUPb13Pl9RALDvQowooTmaur8wD954sYG7GfCooCEzOgvw1431xH8qooYblQiL8YcIW3CXnl8oPYmxmnBTFtZ1kWAWeKr0iHjHJdwC23QxLAplQk_qwDs7hTEWXaAQhLsLZB5NVpDVljB_TkpnHa1uOAUcfmblCPO9qNUDW6fgupM4IvBwvcUf.3atahFLE_5b3eHOOiOUseZVacN7XxWUML8ci_8_vnd1xPomPHvXKmGpmxeh51CVotFzvLpmpLd8NAYp5xDm25LOv16Swq3KpjKHr.XSrtf4yJj8fFJ0tzaD_uyPb1HAzGr.BBICSRcewkJbtfVgYHp1pADx2MyY8FflvE3cvvuRfX_T7PuApSpaeinEvlaMJozePwO75m8oNOej6_JdmvqpO3PSjvttccp4Mc14QZuuw3gI.Gofc_Zk6RSPKXA2gZZ7zpKFb5fZ6LaIvbgexEI68JWZGT7RrNOr5p3PNyQp6Q0o17YK8aP8C4QCaWALGhowfx5te.8sIfYi6zQyXx_90DYTFbifQ0iSMrc03DObTfxyHtKXSsfrBioaBWpI3lG1nlYMrf.vMAFgIRifzY_5d0RIzaAZWrtMUX7_bFwuLCQhiojVeTE5QWY5n8wTX2FdrHmtExAig.Z5Hc-


In [87]:
def get_league():
    league_key = f"{game_id}.l.{league_id}"
    url = f"{BASE_URL}/league/{league_key}"
    resp_data = api_request(url)["league"]
    return resp_data

In [88]:
def get_all_team_rosters_by_date(date):
    """
    Parameters:
    date (str): Date to get roster for in YYYY-MM-DD format

    Returns:
    List of teams with the roster details for that date
    (does not include stats, need to make a separate reequest to get stats for all these players)
    Trieed everything, can't get stats if you're getting data for multiple teams
    f"{BASE_URL}/league/{league_key}/teams/roster;date={date}/players/stats;type=date;date={date}" doesn't work
    """
    league_key = f"{game_id}.l.{league_id}"
    url = f"{BASE_URL}/league/{league_key}/teams/roster;date={date}/players" # TODO: Would it be better to store team_keys once and make request to teams endpoint instad of league?
    resp_data = api_request(url)["league"]["teams"]
    return data

In [92]:
def api_request(url):
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(response.text)
        response.raise_for_status()
    xml = response.text
    data = xml_to_dict(response.text)["fantasy_content"]
    return data

In [90]:
def get_stats_for_players_by_date(player_keys, date):
    """
    Parameters:
    player_keys (str[]): All player keys
    date (str): Date to get roster for in YYYY-MM-DD format
    """
    league_key = f"{game_id}.l.{league_id}"
    data = []
    for i in range(math.ceil(len(player_keys)/25)):
        player_keys_str = ",".join(player_keys[i*25:min((i+1)*25, len(player_keys))])
        url = f"{BASE_URL}/players;player_keys={player_keys_str}/stats;type=date;date={date};start={i*25}"
        resp_data = api_request(url)
        data += resp_data["players"]
    return data

In [93]:
"""
Selected Position Values: {'G', 'NA', 'IR+', 'D', 'BN', 'RW', 'LW', 'C'}
- Number of games played by team
- Points left on bench
"""
league = get_league()
league_start_date = datetime.strptime(league["start_date"], "%Y-%m-%d")
league_end_date = datetime.strptime(league["end_date"], "%Y-%m-%d")
games_by_team = defaultdict(int)
for i in range((league_end_date - league_start_date).days+1):
    print(i)
    current_date_str = (league_start_date + timedelta(days=i)).strftime("%Y-%m-%d")
    teams = get_all_team_rosters_by_date(current_date_str)
    player_keys = [player["player_key"] for team in teams for player in team["roster"]["players"]] # TODO: Filter out injured players?
    players = get_stats_for_players_by_date(player_keys, current_date_str)
    players_playing = [player["player_key"] for player in players if player["player_stats"]["stats"][0]["value"] != "-"] # Filter out players that didn't play that day
    for team in teams:
        for player in team["roster"]["players"]:
            if player["selected_position"]["position"] not in ["BN", "IR+", "NA"] and player["player_key"] in players_playing:
                games_by_team[team["team_key"]] += 1
games_by_team_sorted = sorted(games_by_team.items(), key=lambda item: item[1], reverse=True)
print(games_by_team_sorted)

<?xml version='1.0' encoding='UTF-8'?>
<yahoo:error xmlns:yahoo='http://yahooapis.com/v1/base.rng'
  xml:lang='en-US'>
  <yahoo:description>Please provide valid credentials. OAuth oauth_problem="token_expired", realm="yahooapis.com"</yahoo:description>
</yahoo:error>



HTTPError: 401 Client Error: Unauthorized for url: https://fantasysports.yahooapis.com/fantasy/v2/league/427.l.97108

In [ ]:
def get_league_game_weeks():
    game_weeks = query.query(
        f"https://fantasysports.yahooapis.com/fantasy/v2/game/{game_id}/game_weeks", 
        ["game", "game_weeks"]
    )
    game_weeks_league = game_weeks
    game_weeks_league[league_start_week-1].start = league_start_date_str
    game_weeks_league[league_end_week-1].end = league_end_date_str
    return game_weeks_league

In [ ]:
league_game_weeks = get_league_game_weeks()

In [ ]:
def get_dates_by_week(week):
    start_date_str = league_game_weeks[week-1].start
    end_date_str = league_game_weeks[week-1].end
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    return [(start_date + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end_date - start_date).days + 1)]

In [ ]:
def get_players(player_keys):
    return [
        player 
        for i in range(int(len(player_keys) / 25) + 1)
        for player in query.query(
            f"https://fantasysports.yahooapis.com/fantasy/v2/league/{league_info.league_key}/players;player_keys={','.join(player_keys[i*25:min((i+1)*25, len(player_keys))])};start={i*25}/stats",
            ["league", "players"]
        )
    ]

In [ ]:
def get_top_n_players_by_position(n, position):
    if position == "F":
        position = "C,LW,RW"
    return [playere for i in range(int(n/25)+1) for player in query.query(
        f"https://fantasysports.yahooapis.com/fantasy/v2/league/{league_info.league_key}/players;sort=PTS;sort_type=season;position={position};count={n};start={i*25}/stats",
        ["league", "players"]
    )]

In [ ]:
'''
Get's daily scores for each matchup of the week.
'''
def get_matchups_by_week(week):
    dates = get_dates_by_week(week)
    resp = query.get_response(
        f"https://fantasysports.yahooapis.com/fantasy/v2/league/{league_info.league_key}/scoreboard;week={week}/matchups/teams/stats_collection;types=date;date={','.join(dates)}", 
    )
    teams_dict = resp.json().get('fantasy_content', {}).get('league', [{}])[1].get('scoreboard', {}).get("0").get("matchups", {})
    teams_dict = unpack_data(teams_dict, YahooFantasyObject)
    teams_dict = [el["matchup"] for el in teams_dict] # Convert object to list
    return teams_dict

In [ ]:
def get_player_game_log_nhl(player_id):
    season = f"{league_info.season}{league_info.season+1}"
    return requests.get(f"https://api-web.nhle.com/v1/player/{player_id}/game-log/{season}/2").json()["gameLog"] # 2 inndicates regular season games

In [ ]:
def get_game_info_nhl(game_id):
    return requests.get(f"https://api-web.nhle.com/v1/gamecenter/{game_id}/landing").json()


In [ ]:
def get_player_team_by_date(player_name: str, player_position: str, date: str):
    '''
    Parameters:
    player_name (str): Full name of player
    date (str): Date to search for in YYYY-MM-DD format

    Returns:
    str: 3 letter abbreviation for NHL team name
    '''
    player_name_url = player_name.replace(" ", "%20")
    players = requests.get(f"https://search.d3.nhle.com/api/v1/search/player?culture=en-us&limit=20&q={player_name_url}%2A").json()
    players = [player for player in players if player["name"] == player_name] # Matching names
    # If there are multiple players matching name, filter by other attributes
    if len(players) > 1:
        players = [player for player in players if player["lastSeasonId"] and int(player["lastSeasonId"][:4]) >= league_info.season]
        if len(players) > 1:
            players = [player for player in players if player["positionCode"] in player_position.split(",")]
    if len(players) != 1:
        print(len(players), player_name, players)
    player = players[0]
    player_game_log = get_player_game_log_nhl(player["playerId"])
    game_log_for_date = next([game_log for game_log in player_game_log if game_log["gameDate"] == date])
    print(game_log_for_date)
    # return team_name

In [ ]:
def get_player_team_on_date(player_game_log, date):
    game = next(iter([game for game in player_game_log if datetime.strptime(game["gameDate"], "%Y-%m-%d") <= datetime.strptime(date, "%Y-%m-%d")]), None)
    if game is None:
        team = player_game_log[-1]["teamAbbrev"]
    else:
        team = game["teamAbbrev"]
    return team

In [ ]:
game_logs_cache = {}

In [ ]:
def normalize_name(name):
    # Normalize the name to NFKD form and remove diacritics
    return ''.join(
        c for c in unicodedata.normalize('NFKD', name)
        if not unicodedata.combining(c)
    )

In [ ]:
def get_game_log_by_player(player_key, player_name, player_position):
    if player_key in game_logs_cache:
        return game_logs_cache[player_key]
    player_name_url = player_name.replace(" ", "%20").replace("-", "%20")
    players_resp = requests.get(f"https://search.d3.nhle.com/api/v1/search/player?culture=en-us&limit=20&q={player_name_url}%2A").json()
    players = [player for player in players_resp if normalize_name(player["name"]) == player_name] # Matching names
    # If there are multiple players matching name, filter by other attributes
    if len(players) > 1:
        players = [player for player in players if player["lastSeasonId"] and int(player["lastSeasonId"][:4]) >= league_info.season]
        if len(players) > 1:
            players = [player for player in players if player["positionCode"] in player_position.split(",")]
    if len(players) != 1:
        print(len(players), player_name, players)
    player = players[0]
    player_id = player["playerId"]
    player_game_log = get_player_game_log_nhl(player_id)
    game_logs_cache[player_key] = player_game_log
    return player_game_log

In [ ]:
query.get_game_stat_categories_by_game_id(427)

In [ ]:
'''
NHL Team contributing most to each fantasay team
Doesn't work, player_points doesn't take into account whether player was benched that day:
https://fantasysports.yahooapis.com/fantasy/v2/team/427.l.97108.t.1/roster;date={date}/players/stats;type=date;date={date}
Miami Steamrollers          EDM 0.33
adam2                       TBL 0.26
eric's Awe-Inspiring Team   VAN 0.17
Theodore's Poo Poo Hospital FLA 0.16
Cyrus's Cursed Team         FLA 0.15
Kevin's Incredible Team     BOS 0.15
Tim's Terrific Team         VAN 0.13
Eric’s a Bitch Team         COL 0.12

Eric’s a Bitch Team         Nathan MacKinnon    0.12
adam2                       Nikita Kucherov     0.11
Miami Steamrollers          Evan Bouchard       0.11
Kevin's Incredible Team     David Pastrnak      0.11
Theodore's Poo Poo Hospital Connor McDavid      0.10
eric's Awe-Inspiring Team   J.T. Miller         0.10
Tim's Terrific Team         Auston Matthews     0.09
Cyrus's Cursed Team         Matthew Tkachuk     0.08

Tim's Terrific Team         1426
Cyrus's Cursed Team         1061
Theodore's Poo Poo Hospital 985
eric's Awe-Inspiring Team   827
Eric’s a Bitch Team         774
Miami Steamrollers          711
adam2                       599
Kevin's Incredible Team     433
'''
# TODO: Properly iterate through teams
HITS_STAT_ID = 31
hits_by_team = defaultdict(int)
for team in league_teams:
    team_points_by_nhl_team = defaultdict(float)
    team_points_by_player = defaultdict(float)
    for week in range(league_start_week, league_end_week + 1):
        week_end_date = get_dates_by_week(week)[-1]
        roster_players = query.query(f"https://fantasysports.yahooapis.com/fantasy/v2/team/{team.team_key}/roster;week={week}/players/stats;type=week;week={week}", ["team", "roster", "0", "players"])
        for player in roster_players:
            if not player.player_points.total:
                continue
            player_game_log = get_game_log_by_player(player.player_key, player.name.full, player.display_position)
            nhl_team = get_player_team_on_date(player_game_log, week_end_date)
            print(player)
            team_points_by_nhl_team[nhl_team] += player.player_points.total
            team_points_by_player[player.player_key] += player.player_points.total
            hits_by_team[team.team_key] += next(iter([stat.value for stat in player.player_stats.stats if stat.stat_id == HITS_STAT_ID]), 0)
    team_points_by_nhl_team = sorted(team_points_by_nhl_team.items(), key=lambda item: item[1], reverse=True)
    team_points_by_player = sorted(team_points_by_player.items(), key=lambda item: item[1], reverse=True)
    top_nhl_team = team_points_by_nhl_team[0][0]
    top_nhl_team_pct = round(team_points_by_nhl_team[0][1]/sum([row[1] for row in team_points_by_nhl_team]), 2)
    top_player = team_points_by_player[0][0]
    top_player_name = query.query(f"https://fantasysports.yahooapis.com/fantasy/v2/player/{top_player}", ["player"], Player).name.full
    top_player_pct = round(team_points_by_player[0][1]/sum([row[1] for row in team_points_by_player]), 2)
    print(team.name.decode('utf-8'), top_nhl_team, top_nhl_team_pct)
    print(team.name.decode('utf-8'), top_player_name, top_player_pct)
    print(team.name.decode('utf-8'), hits_by_team[team.team_key])

In [ ]:
# hits_by_team = sorted(hits_by_team.items(), key=lambda item: item[1], reverse=True)
test = [print(next(team.name.decode('utf-8') for team in league_teams if team.team_key == team_key), round(hits)) for (team_key, hits) in hits_by_team]

In [ ]:
'''
Most/least points after dropping a player on average
'''
transactions = query.get_league_transactions()
transactions_drop = [{"player": player, "timestamp": transaction.timestamp} for transaction in transactions for player in transaction.players if player.transaction_data.type == "drop"]
team_name = {}
points_by_team = defaultdict(list)
for transaction in transactions_drop:
    print(transaction)
    transaction_date = datetime.fromtimestamp(transaction["timestamp"], tz=ZoneInfo("America/New_York"))
    transaction_player = transaction["player"]
    team_key = transaction_player.transaction_data.source_team_key
    player_name = transaction_player.name.full
    player_name_url = player_name.replace(" ", "%20")
    players = requests.get(f"https://search.d3.nhle.com/api/v1/search/player?culture=en-us&limit=20&q={player_name_url}%2A").json()
    players = [player for player in players if player["name"] == player_name] # Matching names
    # If there are multiple players matching name, filter by other attributes
    if len(players) > 1:
        players = [player for player in players if player["lastSeasonId"] and int(player["lastSeasonId"][:4]) >= league_info.season]
        if len(players) > 1:
            players = [player for player in players if player["positionCode"] in transaction_player.display_position.split(",")]
    if len(players) != 1:
        print(len(players), player_name, players)
    player = players[0]
    player_game_log = get_player_game_log_nhl(player["playerId"])
    if len(player_game_log) == 0:
        print(f"{player_name} didn't play any games this season")
        continue
    # TODO: Consider case when someone is dropped due to injury
    # last_game = [game_log for game_log in player_game_log if datetime.strptime(game_log["gameDate"], "%Y-%m-%d").replace(tzinfo=timezone(timedelta(hours=-4))).date() <= transaction_date.date()]
    # if len(last_game) == 0: # No games played before drop
    #     first_game = player_game_log[-1]
    #     print(f"{player_name} didn't play any games yet")
    #     continue
    # Next game could either be the same day as pickup or game after pickup
    next_game = [game_log for game_log in player_game_log if datetime.strptime(game_log["gameDate"], "%Y-%m-%d").replace(tzinfo=timezone(timedelta(hours=-4))).date() >= transaction_date.date()]
    if len(next_game) == 0: # Didn't play the rest of the season
        print(f"{player_name} didn't play any more games")
        continue
    next_game_date = next_game[-1]["gameDate"]
    player_stats = query.get_player_stats_by_date(transaction_player.player_key, next_game_date)
    points_by_team[team_key].append(player_stats.player_points.total)
    team_name[team_key] = transaction_player.transaction_data.source_team_name
team_average = []
for team_key, team_points in points_by_team.items():
    team_average.append((team_name[team_key], round(sum(team_points)/len(team_points), 1)))
team_average = sorted(team_average, key=lambda x: x[1])
print(team_average)

In [ ]:
'''
Biggest comeback
TODO: Add the following line to bottom of Team class:
self.team_stats_collection: List[TeamPoints] = self._extracted_data.get("team_stats_collection", []) # ADDED
52.8s
'''
deficits = []
for week in range(league_start_week, league_end_week + 1):
    matchups = get_matchups_by_week(week)
    for matchup in matchups:
        # Comeback win can't happen without a winner
        if matchup.is_tied:
            continue
        winner_id = int(matchup.winner_team_key.split(".")[-1])
        team_ids = [matchup.teams[0].team_id, matchup.teams[1].team_id]
        team_w_idx, team_l_idx = (0, 1) if winner_id == team_ids[0] else (1, 0)
        deficit = 0
        start_date = datetime.strptime(matchup.week_start, "%Y-%m-%d")
        end_date = datetime.strptime(matchup.week_end, "%Y-%m-%d")
        # Don't include the last day since the matchup is over
        for i in range((end_date - start_date).days):
            current_date_str = (start_date + timedelta(days=i)).strftime("%Y-%m-%d")
            team_w_points = matchup.teams[team_w_idx].team_stats_collection[i]["team_points"].total
            team_l_points = matchup.teams[team_l_idx].team_stats_collection[i]["team_points"].total
            deficit = round(deficit + team_l_points - team_w_points, 1)
            if deficit > 0:
                deficits.append((deficit, matchup))
biggest_deficits = sorted(deficits, key=lambda x: x[0])
biggest_deficits = list({f"{matchup.week}.{matchup.winner_team_key}": (deficit, matchup) for (deficit, matchup) in biggest_deficits}.values())
biggest_deficits = sorted(biggest_deficits, reverse=True, key=lambda x: x[0])
print("Biggest comebacks")
for diff, matchup in biggest_deficits[:5]:
    print(f"{matchup.week} {matchup.teams[0].name} {matchup.teams[1].name} - {diff}")

In [ ]:
'''
Biggest draft busts/steals
'''
# Get draft results
draft_results = query.get_league_draft_results()
draft_players = get_players([draft_result.player_key for draft_result in draft_results])
draft_players
draft_players_by_pos = {"F": [], "D": [], "G": []}
positions_map = {"C": "F", "LW": "F", "RW": "F", "D": "D", "G": "G"}
for draft_player in draft_players:
    draft_players_by_pos[positions_map[draft_player.primary_position]].append(draft_player)
# Get the top players by position
top_players_by_pos = {
    "F": get_top_n_players_by_position(len(draft_players_by_pos["F"]), "F"), 
    "D": get_top_n_players_by_position(len(draft_players_by_pos["D"]), "D"), 
    "G": get_top_n_players_by_position(len(draft_players_by_pos["G"]), "G")
}
# Get differences between draft player and top player
diffs = []
for key in draft_players_by_pos:
    for draft_player, top_player in zip(draft_players_by_pos[key], top_players_by_pos[key]):
        diff = round(draft_player.player_points.total - top_player.player_points.total, 1)
        diffs.append((diff, draft_player))
biggest_diffs = sorted(diffs, reverse=True, key=lambda x: x[0])
smallest_diffs = sorted(diffs, key=lambda x: x[0])
print("Biggest draft steals")
for diff, player in biggest_diffs[:5]:
    print(f"{player.name.first} {player.name.last} - {diff}")
print("Biggest draft busts")
for diff, player in smallest_diffs[:5]:
    print(f"{player.name.first} {player.name.last} - {diff}")